<a href="https://colab.research.google.com/github/gonza77/SQL_ECommerce/blob/main/SQL_ecommerce_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#SQL SERVER CODE, using management studio 19

#Explore tables
SELECT TOP (100) *
FROM [E-commerce ].[dbo].[customers]

--Explore order_details
SELECT TOP (100)*
FROM [E-commerce ].[dbo].[order_details]

--explore orders
SELECT TOP (100)*
FROM [E-commerce ].[dbo].[orders]

--Explore payments

SELECT TOP (100)*
FROM [E-commerce ].[dbo].[payments]

--Explore products
SELECT TOP (100)*
FROM [E-commerce ].[dbo].[products]
where category = 'picture'

--BASIC Queries Exploration, shipped orders
--1
SELECT * FROM [E-commerce ].[dbo].[Orders]
WHERE OrderStatus = 'Shipped';

--2
SELECT * FROM [E-commerce ].[dbo].[products]
ORDER BY Price DESC;

--3 JOINS Exploration

SELECT * FROM [E-commerce ].[dbo].[Orders]
JOIN [E-commerce ].[dbo].[customers] ON Orders.CustomerID = Customers.ID;

--average price and quantity items by product ID
SELECT ProductID, AVG(Price) as average_price, SUM(Quantity) as quantity
FROM [E-commerce ].[dbo].[order_details]
GROUP BY ProductID
order by quantity desc;


-- The average price of the product calculated across all orders, window functions
SELECT ProductID, Price,
       AVG(Price) OVER(PARTITION BY ProductID) AS AvgPricePerProduct
FROM [E-commerce ].[dbo].[Order_Details];

--4 Multiple joins, clients who paid with debit card an amount between 400-500, and the order status is pending
SELECT c.Name, c.Email, p.Amount, p.PaymentMethod, o.OrderStatus
FROM [E-commerce].[dbo].[customers] c
JOIN [E-commerce].[dbo].[orders] o ON c.ID = o.CustomerID
JOIN [E-commerce].[dbo].[payments] p ON o.ID = p.OrderID
WHERE p.PaymentMethod = 'Debit Card'
    AND p.Amount BETWEEN 400 AND 500
    AND o.OrderStatus = 'Pending';

----how many clients are there for each distinct product category
SELECT Category, COUNT(DISTINCT Name) AS PeopleCount
FROM [E-commerce].[dbo].[products]
GROUP BY Category
order by PeopleCount Desc;

--Top 5 Selling Products:
SELECT top(5) p.Name, SUM(od.Quantity) AS TotalSold
FROM [E-commerce].[dbo].[order_details] od
JOIN [E-commerce].[dbo].[products] p ON od.ProductID = p.ID
GROUP BY p.Name
ORDER BY TotalSold DESC


-- Top 5 Customers in Terms of Purchase Amount
SELECT TOP 5 c.Name, SUM(od.Quantity * od.Price) AS TotalSpent
FROM [E-commerce].[dbo].[customers] c
JOIN [E-commerce].[dbo].[orders] o ON c.ID = o.CustomerID
JOIN [E-commerce].[dbo].[order_details] od ON o.ID = od.OrderID
GROUP BY c.Name
ORDER BY TotalSpent DESC;

--Monthly Sales Growth
WITH MonthlySales AS (
    SELECT FORMAT(OrderDate, 'yyyy-MM') AS Month, SUM(od.Quantity * od.Price) AS Sales
    FROM [E-commerce].[dbo].[orders] o
    JOIN [E-commerce].[dbo].[order_details] od ON o.ID = od.OrderID
    GROUP BY FORMAT(OrderDate, 'yyyy-MM')
)
SELECT
    Month,
    Sales,
    (Sales - LAG(Sales) OVER(ORDER BY Month)) / LAG(Sales) OVER(ORDER BY Month) * 100 AS MoMGrowth
FROM MonthlySales
ORDER BY Month Desc;


--Inventory stock SI
SELECT SUM(Price * Stock) AS TotalInventoryValue
FROM [E-commerce].[dbo].[products];


--see all customers that paid with debit card and what products they ordered
SELECT c.Name AS CustomerName, c.Email, p.Name AS ProductName, od.Quantity, pay.Amount, pay.PaymentMethod
FROM [E-commerce].[dbo].[customers] c
JOIN [E-commerce].[dbo].[orders] o ON c.ID = o.CustomerID
JOIN [E-commerce].[dbo].[order_details] od ON o.ID = od.OrderID
JOIN [E-commerce].[dbo].[products] p ON od.ProductID = p.ID
JOIN [E-commerce].[dbo].[payments] pay ON o.ID = pay.OrderID
WHERE pay.PaymentMethod = 'Debit Card';


--number of days until next order for each client
WITH OrderDates AS (
    SELECT
        c.Name AS CustomerName,
        o.OrderDate,
        LEAD(o.OrderDate) OVER(PARTITION BY o.CustomerID ORDER BY o.OrderDate) AS NextOrderDate
    FROM
        [E-commerce].[dbo].[customers] c
        JOIN [E-commerce].[dbo].[orders] o ON c.ID = o.CustomerID
)
SELECT
    CustomerName,
    OrderDate,
    NextOrderDate,
    DATEDIFF(day, OrderDate, NextOrderDate) AS DaysUntilNextOrder
FROM
    OrderDates
ORDER BY

    CustomerName,
	Orderdate
    ;



--advanced
--analyze the total sales per product category and compare it with the overall average sales per order.
WITH SalesCTE AS (
    SELECT
        p.Category,
        od.OrderID,
        od.ProductID,
        od.Quantity,
        od.Price,
        (od.Quantity * od.Price) AS TotalPrice
    FROM [E-commerce ].[dbo].[order_details] od
    JOIN [E-commerce ].[dbo].[products] p ON od.ProductID = p.ID
)
, CategorySalesCTE AS (
    SELECT
        Category,
        SUM(TotalPrice) AS CategoryTotalSales,
        COUNT(DISTINCT OrderID) AS CategoryTotalOrders,
        AVG(SUM(TotalPrice)) OVER() AS OverallAvgSalesPerOrder
    FROM SalesCTE
    GROUP BY Category
)
SELECT
    Category,
    CategoryTotalSales,
    CategoryTotalSales / CategoryTotalOrders AS AvgSalesPerOrderInCategory,
    OverallAvgSalesPerOrder,
    CASE
        WHEN (CategoryTotalSales / CategoryTotalOrders) > OverallAvgSalesPerOrder THEN 'Above Average'
        ELSE 'Below Average'
    END AS SalesComparison
FROM CategorySalesCTE
ORDER BY CategoryTotalSales DESC;

